## 평균 구해보기

- mapValues : 값만 바꾼다. 키는 만지지않음
- `rdd.mapValues(lambda v: 새로운_v)`

In [1]:
from pyspark.sql import SparkSession

In [4]:
spark=(
    SparkSession
    .builder
    .appName("avg")
    .master("spark://spark-master:7077")
    .getOrCreate()
)

In [24]:
sc=spark.sparkContext

In [25]:
# mapValues()확인 연습예제

In [13]:
data = [("Seoul", 1000), ("Seoul", 2000)]

In [17]:
rdd=sc.parallelize(data)
rdd.collect()

[('Seoul', 1000), ('Seoul', 2000)]

In [22]:
new_rdd=rdd.mapValues(lambda price:(price,1))
new_rdd.collect()

[('Seoul', (1000, 1)), ('Seoul', (2000, 1))]

In [26]:
# reduceByKey() 확인 연습 

In [27]:
from functools import reduce
values = [(10,1), (7,1), (4,1)]

def check_reduce(a,b):
    print(f"a:{a},b:{b}")
    return (a[0]+b[0],a[1],b[1])
reduce(check_reduce,values)

a:(10, 1),b:(7, 1)
a:(17, 1, 1),b:(4, 1)


(21, 1, 1)

In [21]:
# 평균 연습

In [6]:
csv_file=sc.textFile("file:///workspace/data/house_price.csv")
csv_file.take(2)

['seoul,10000,3', 'seoul,10000,5']

In [8]:
def parse_line(line:str):
    city,price,count=line.split(",")
    return (int(price),int(count))

In [10]:
price_count=csv_file.map(parse_line)
price_count.take(2)

[(10000, 3), (10000, 5)]

In [29]:
sum_of_count=(
    price_count
    .mapValues(lambda v:(v,1))
    .reduceByKey(lambda a,b:(a[0]+b[0],a[1]+b[1]))
)
sum_of_count.collect()

[(10000, (8, 2)),
 (40000, (7, 1)),
 (5000, (14, 2)),
 (4000, (4, 2)),
 (9000, (4, 1)),
 (8000, (9, 1))]

In [33]:
avg_by_count=(
    sum_of_count
    .mapValues(lambda total_count:int(total_count[0]/total_count[1]))
)

In [37]:
for price, avg_count in avg_by_count.collect():
    print(f"집값 {price}인 데이터의 count 평균은 {avg_count} 입니다.")

집값 10000인 데이터의 count 평균은 4 입니다.
집값 40000인 데이터의 count 평균은 7 입니다.
집값 5000인 데이터의 count 평균은 7 입니다.
집값 4000인 데이터의 count 평균은 2 입니다.
집값 9000인 데이터의 count 평균은 4 입니다.
집값 8000인 데이터의 count 평균은 9 입니다.


In [38]:
spark.stop()